In [14]:
! pip install pandas \
    matplotlib

In [15]:
import pandas as pd
import glob as gb
from pprint import pprint
from io import StringIO

In [16]:
DIR = 'airflow/logs/dag_id=app_template/*/*/*'
logs_paths = gb.glob(DIR)

df_logs = pd.DataFrame()
for path in logs_paths:

    with open(path, 'r') as f:
        lines = f.readlines()
        lines = [l for l in lines if '{helpers.py:110}' in l]
        lines = ''.join(lines)

    df = pd.read_csv(StringIO(lines), sep=' ', header=None)
    df = df[df[1]=='{helpers.py:110}']
    col_names = {
        0: 'timestamp',
        4: 'tasks',
        5: 'batch_id',
        6: 'processed_%',
        7: 'seconds'
    }
    df = df[col_names.keys()]
    df.rename(columns=col_names, inplace=True)
    task_type = path.split('/')[4].split('=')[1].split('.')[0]
    df['task_type'] = task_type
    run_id = path.split('/')[3].split('=')[1].split('__')[1]
    df['run_id'] = run_id
    df_logs = pd.concat([df_logs, df], ignore_index=True)
df_logs.reset_index(drop=True, inplace=True)

df_logs['seconds'] = pd.to_numeric(df_logs['seconds'], errors='coerce')
df_logs['processed_%'] = pd.to_numeric(df_logs['processed_%'], errors='coerce')
df_logs['timestamp'] = \
    pd.to_datetime(df_logs['timestamp'].str.strip('[]'), errors='coerce')

df_logs.to_csv('tmp/logs.csv', index=False)

display(df_logs.shape)
df_logs.sample(10)

(46575, 7)

,timestamp,tasks,batch_id,processed_%,seconds,task_type,run_id
19429,2025-06-08 19:05:07.394000+00:00,2-2-5,30,48.39,1.768,extractors,74bb7ddd-d302-43d0-8fa1-3872f2708aea
42160,2025-06-08 22:49:05.804000+00:00,4-3-5,30,73.17,3.087,transformers,db3f7df9-2f15-43f0-a3e1-c33422e7edcb
4226,2025-06-08 17:28:01.697000+00:00,1-4-2,29,93.55,3.268,transformers,6b547fb1-9bf6-4ae6-b881-effdf9a3b431
9144,2025-06-08 22:26:51.662000+00:00,4-2-4,19,61.29,2.477,extractors,b1075bb8-0ce0-4be0-ac65-d7f20e3a6dea
14824,2025-06-08 21:22:16.230000+00:00,3-5-1,27,65.85,2.500,extractors,18d896ea-591b-4d33-b5dc-316440c0ac31
26531,2025-06-08 20:08:46.007000+00:00,3-1-3,36,87.80,2.253,extractors,18257c7a-4fe5-4fc4-bc42-2300c43f83c1
2584,2025-06-08 17:48:08.709000+00:00,1-5-1,98,78.40,1.541,extractors,5a6ab6bb-e22e-4832-aa17-c31220756729
22443,2025-06-08 22:44:54.728000+00:00,4-3-4,15,48.39,2.018,extractors,32423878-f2df-4d75-b920-332a0bacc868
24920,2025-06-08 21:17:51.447000+00:00,3-4-4,27,87.10,2.200,loaders,a62faef2-93e4-49a3-b682-7d28db45895d
4382,2025-06-08 17:27:17.166000+00:00,1-4-2,30,24.00,2.009,extractors,6b547fb1-9bf6-4ae6-b881-effdf9a3b431
